<a href="https://colab.research.google.com/github/Kimhansav/introofnlp_practice/blob/main/14_01(%EC%8B%9C%ED%80%80%EC%8A%A4_%ED%88%AC_%EC%8B%9C%ED%80%80%EC%8A%A4(Sequence_to_Sequence%2C_seq2seq))practice_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def download_zip(url, output_path):
  response = requests.get(url, headers = headers, stream = True)
  if response.status_code == 200:
    with open(output_path, 'wb') as f:
      for chunk in response.iter_content(chunk_size = 8192):
        f.write(chunk)
    print(f"ZIP file downloaded to {output_path}")
  else:
    print(f"Failed to download. HTTP Response Code: {response.status_code}")

url = "http://www.manythings.org/anki/fra-eng.zip"
output_path = "fra-eng.zip"
download_zip(url, output_path)

path = os.getcwd()
zipfilename = os.path.join(path, output_path)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
  zip_ref.extractall(path)

ZIP file downloaded to fra-eng.zip


In [ ]:
lines = pd.read_csv('fra.txt', names = ['src', 'tar', 'lic'], sep = '\t')
del lines['lic']
print('전체 샘플의 개수 :', len(lines))

전체 샘플의 개수 : 229803


In [ ]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] #6만개만 저장
lines.sample(10)

,src,tar
30354,We abhor violence.,Nous abhorrons la violence.
40079,He stopped drinking.,Il a arrêté de boire.
42564,It's very hot today.,Il fait vraiment chaud aujourd'hui.
41465,I want my life back.,Je veux qu'on me rende ma vie.
38291,You can talk later.,Vous pouvez discuter plus tard.
54718,Are you on medication?,Est-ce que tu prends des médicaments ?
7641,I am homesick.,J'ai le mal du pays.
3671,I'm unhappy.,Je suis mécontente.
5983,Let Tom know.,Informe Tom.
29402,They arrested her.,Ils l'ont arrêtée.


In [ ]:
lines.tar = lines.tar.apply(lambda x: '\t ' + x + ' \n')
lines.sample(10)

,src,tar
38817,You're very clever.,\t Vous êtes très ingénieuses. \n
43163,She kept on working.,\t Elle continua à travailler. \n
44290,Tom has gone insane.,\t Tom a perdu les pédales. \n
4946,Get a doctor.,\t Va chercher un médecin ! \n
29370,There's no change.,\t Il n'y a pas de changement. \n
25990,He has a few pens.,\t Il a quelques stylos. \n
7641,I am homesick.,\t J'ai le mal du pays. \n
58809,It's a riveting story.,\t C'est une histoire fascinante. \n
40918,I just don't buy it.,"\t Je ne marche pas, un point c'est tout. \n"
21941,It's a wire cage.,\t Cette cage est en fil de fer. \n


In [ ]:
#문자 집합 구축
src_vocab = set()
for line in lines.src: #1줄씩 읽음
  for char in line: #1개의 문자씩 읽음
    src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
  for char in line:
    tar_vocab.add(char)

In [ ]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 80
target 문장의 char 집합 : 104


In [ ]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x']


In [ ]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, 'ï': 77, '’': 78, '€': 79}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 3

In [ ]:
encoder_input = []

#1개의 문장
for line in lines.src:
  encoded_line = []
  #각 줄에서 1개의 char
  for char in line:
    #각 char을 정수로 변환
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩 :', encoder_input[:5])

source 문장의 정수 인코딩 : [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


In [ ]:
decoder_input = []
for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :', decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 3, 48, 52, 3, 4, 3, 2], [1, 3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [1, 3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [1, 3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [1, 3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [ ]:
#이제 디코더의 예측값과 비교할 실제값을 인코딩해야 하는데, 이때 인코딩되는 문장에는 시작 단어인 <sos>가 필요없으므로 문장에서 이를 제거한 뒤 처리한다.

decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :', decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 48, 52, 3, 4, 3, 2], [3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [ ]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :', max_src_len)
print('target 문장의 최대 길이 :', max_tar_len)

source 문장의 최대 길이 : 22
target 문장의 최대 길이 : 76


In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen = max_src_len, padding = 'post')
decoder_input = pad_sequences(decoder_input, maxlen = max_tar_len, padding = 'post')
decoder_target = pad_sequences(decoder_target, maxlen = max_tar_len, padding = 'post')

In [ ]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
encoder_inputs = Input(shape = (None, src_vocab_size))
encoder_lstm = LSTM(units = 256, return_state = True)

#encoder_outputs는 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

#LSTM은 바닐라 RNN과는 달리 상태가 두 개, 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape = (None, tar_vocab_size))
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state = True)

#디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation = 'softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer = "rmsprop", loss = "categorical_crossentropy")

In [ ]:
model.fit(x = [encoder_input, decoder_input], y = decoder_target, batch_size = 64, epochs = 40, validation_split = 0.2)

Epoch 1/40
750/750 [==============================] - 17s 17ms/step - loss: 0.8559 - val_loss: 0.7730
Epoch 2/40
750/750 [==============================] - 10s 13ms/step - loss: 0.5752 - val_loss: 0.6686
Epoch 3/40
750/750 [==============================] - 10s 13ms/step - loss: 0.5063 - val_loss: 0.6060
Epoch 4/40
750/750 [==============================] - 10s 14ms/step - loss: 0.4606 - val_loss: 0.5542
Epoch 5/40
750/750 [==============================] - 10s 14ms/step - loss: 0.4253 - val_loss: 0.5228
Epoch 6/40
750/750 [==============================] - 10s 14ms/step - loss: 0.3983 - val_loss: 0.4971
Epoch 7/40
750/750 [==============================] - 10s 14ms/step - loss: 0.3780 - val_loss: 0.4781
Epoch 8/40
750/750 [==============================] - 10s 13ms/step - loss: 0.3615 - val_loss: 0.4592
Epoch 9/40
750/750 [==============================] - 11s 14ms/step - loss: 0.3477 - val_loss: 0.4444
Epoch 10/40
750/750 [==============================] - 10s 14ms/step - loss: 0.335

In [ ]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)

In [ ]:
#이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape = (256,))
decoder_state_input_c = Input(shape = (256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

#문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
#뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)

#훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs = [decoder_inputs] + decoder_states_inputs, outputs = [decoder_outputs] + decoder_states)

In [ ]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [ ]:
def decode_sequence(input_seq):
  #입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  #<SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  #stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    #이전 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    #예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    #현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    #<eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
      stop_condition = True

    #현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    #현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [ ]:
for seq_index in [3, 50, 100, 300, 1001]: #입력 문장의 인덱스
  input_seq = encoder_input[seq_index : seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index]) - 1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence) - 1]) #'\n'을 빼고 출력

1/1 [==============================] - 0s 22ms/step
-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Décampe ! 
1/1 [==============================] - 0s 18ms/step
-----------------------------------
입력 문장: Hello!
정답 문장: Bonjour ! 
번역 문장: Soyez bon ! 
1/1 [==============================] - 0s 20ms/step
-----------------------------------
입력 문장: Got it!
정답 문장: J'ai pigé ! 
번역 문장: Continue ! 
1/1 [==============================] - 0s 28ms/step
-----------------------------------
입력 문장: Go home.
정답 문장: Rentre à la maison. 
번역 문장: Va au travail essayer. 
1/1 [==============================] - 0s 19ms/step
-----------------------------------
입력 문장: Get going.
정답 문장: En avant. 
번역 문장: Décampe ! 
